# Aggregatie

Met aggragatie bedoelen we het *samenvatten* van de gegevens van een aantal documenten in een enkele waarde.
Voorbeelden van aggregatie-operatoren zijn: `sum` (en daarmee ook `count`), `avg`, `min`, `max`.
(Dit zijn ook typische aggregatie-functies in spreadsheets.)
Dit samenvatten combineren we vaak met *groepering*: we willen de samenvattende waarde per groep weten.

> In SQL heb je voor aggregatie dezelfde functies zoals hierboven genoemd; 
  deze gebruik je in het SELECT-deel van de query. Voor groepering gebruik je GROUP BY.

Voor meer informatie over aggregatie in MongoDB, zie: https://docs.mongodb.com/manual/aggregation

Voor de aggregatie-voorbeelden gebruiken we naast de contacten-collection, ook een collection met agenda-items.

De onderstaande opdracht is nodig als initialisatie in sommige omgevingen.

In [1]:
import os
os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"

De gebruikelijke initialisatie:

In [2]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]

contacts = db.contacts
contacts.drop()
os.system('mongoimport -d ' + dbname + ' -c contacts adressen.json')

Database name: eelco-demodb
Mongo version 3.11.0


0

In [3]:
agenda = db.agenda
agenda.drop()
os.system('mongoimport -d ' + dbname + ' -c agenda agenda.json')

0

In [4]:
cursor = agenda.find()
list(cursor)

[{'_id': ObjectId('5fa945a655cba8618465a2f6'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'email': 'zwartejoop@ziggo.nl'}],
  'location': 'Seats2Meet Utrecht CS'},
 {'_id': ObjectId('5fa945a655cba8618465a2f7'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 3, 19, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com'},
   {'email': 'a34huis@gmail.com'}],
  'location': 'Den Haag CS'},
 {'_id': ObjectId('5fa945a655cba8618465a2f8'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 3, 1, 0, 0),
  'duration': 60,
  'participants': [{'email': 'hdb@example.com'},
   {'email': 'a34huis@gmail.com'}],
  'location': 'Seats2Meet Den Bosch'},
 {'_id': ObjectId('5fa945a655cba8618465a2f9'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 23, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com'},
   {'email': 'a34huis@gmail.com'}],
  'location': 'Seats2Me

## Aggregatie in MongoDB: pijplijn

De `colelction.aggregate`-functie heeft een lijst van parameters (`[ {...stap...}, ...]`).
Elk element is deze lijst is een stap in een *pijplijn*,
waarbij elke stap het resultaat van de vorige stap verder bewerkt.
De eerste stap werkt op alle documenten in de *collection*.

Zie ook de figuur in: https://docs.mongodb.com/manual/aggregation/#aggregation

Als stap kun je onder andere de volgende functies gebruiken:

* `$match`: selecteren (filteren) van documenten
* `$group`: voor het groeperen en samenvatten van documenten
* `$project`: selecteren van velden in de resultaat-documenten
* `$lookup`: combineren van documenten uit verschillende collections ("join").

Er zijn nog veel meer functies mogelijk in de aggregation-pijplijn,
zie het MongoDB manual: https://docs.mongodb.com/manual/meta/aggregation-quick-reference/

De `$lookup` en `$project` functies behandelen we in een volgend notebook.

### Voorbeeld 1

In het eerste voorbeeld tellen we het aantal contacten per plaats.
We laten het `$match` query-document hier weg: we onderzoeken alle documenten in de collection.
We groeperen per plaats (via de `_id`) en geven per plaats het aantal (`count`) dat we berekenen door 1 te sommeren voor alle documenten.

Merk op dat je voor de veldnamen van het oorspronkelijke document **in het $group-deel een `$` schrijft.** Bijvoorbeeld: `$address.city`.

In [5]:
cursor = contacts.aggregate([{"$group":{"_id": "$address.city", "count": {"$sum": 1} }}])
list(cursor)

[{'_id': 'Amsterdam', 'count': 3},
 {'_id': 'Rotterdam', 'count': 3},
 {'_id': None, 'count': 1}]

### Voorbeeld 2

Voor het tweede voorbeeld gebruiken we de agenda-items.
Bekijk eerst het bestand [agenda.json](agenda.json) om een indruk te krijgen van de gegevens in deze items.

We willen weten hoeveel tijd we voor de verschillende onderwerpen (`subject`) in de agenda hebben staan.
Merk op dat voor de veldnamen in het $group-deel een `$` geschreven is: `$subject` en `$duration`.


In [6]:
cursor = agenda.aggregate([{"$group":{"_id": "$subject", "count": {"$sum": "$duration"} }}])
list(cursor)

[{'_id': 'Fusie', 'count': 300}, {'_id': 'Beleidsplan', 'count': 420}]

### Opdracht

Maak een query voor het bepalen van de totale tijd per vergaderlocatie (`location`).

In [7]:
cursor = agenda.aggregate([{"$group":{"_id": "$location", "totaltime": {"$sum": "$duration"} }}])
list(cursor)

[{'_id': 'Seats2Meet Den Bosch', 'totaltime': 180},
 {'_id': 'Amsterdam Zuid', 'totaltime': 120},
 {'_id': 'Den Haag CS', 'totaltime': 300},
 {'_id': 'Seats2Meet Utrecht CS', 'totaltime': 120}]

### Voorbeeld 3

We gebruiken hier het `$match`-deel met een filter-document: we willen alleen de documenten waarvan de `participants` de persoon met email-adres: hdb@example.com bevat.
Merk op dat dit een *normaal* filter-document is, als bij `collection.find`: de namen van de velden (properties) schrijf je hier zonder `$`.

De totale vergadertijd per locatie met de persoon met email-adres: hdb@example.com:

In [8]:
cursor = agenda.aggregate([{"$match": {"participants": { "email": "hdb@example.com"}}}, 
                           {"$group": {"_id": "$location", "count": {"$sum": "$duration"}}} ])
list(cursor)

[{'_id': 'Seats2Meet Den Bosch', 'count': 180},
 {'_id': 'Den Haag CS', 'count': 300}]

### Opdracht

Maak een query voor het bepalen van de totale tijd *per vergaderlocatie*, voor het onderwerp (subject) "Beleidsplan".


In [9]:
cursor = agenda.aggregate([
    {"$match": {"subject": "Beleidsplan"}},
    {"$group":{"_id": "$location", "totaltime": {"$sum": "$duration"} }}])
list(cursor)

[{'_id': 'Amsterdam Zuid', 'totaltime': 120},
 {'_id': 'Seats2Meet Den Bosch', 'totaltime': 180},
 {'_id': 'Seats2Meet Utrecht CS', 'totaltime': 120}]